<a href="https://colab.research.google.com/github/antoniovfonseca/spatialAutocorrelation/blob/main/SpatialCorrelationTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio numpy libpysal esda -q
!pip install geemap earthengine-api -q
!pip install earthengine-api -q
!pip install -q geopandas -q
!pip install -U geemap -q
!pip install -U leafmap -q

In [ ]:
import ee
import time
import leafmap
import geemap
import rasterio
import numpy as np
import geopandas as gpd
import shapely.geometry as sg
from esda.moran import Moran
from google.colab import drive
from libpysal.weights import lat2W

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-antoniovicgf')

In [ ]:
geemap.ee_initialize()

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define anos de interesse
year1 = 1990
year2 = 2020

In [ ]:
# Load MapBiomas Collection 9
mapbiomas = ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_irrigated_agriculture_v1")

# Load Mask
municipalities = ee.Image("projects/nasa-grant/assets/database/municipalitiesRaster").eq(1)

In [ ]:
# Converte valores (divide por 100 como no script JS)
mapbiomas = mapbiomas.divide(100).toInt()

In [ ]:
# Extrai camadas para os anos desejados
irrig_1990 = mapbiomas.select(f'irrigated_agriculture_{year1}')
irrig_2020 = mapbiomas.select(f'irrigated_agriculture_{year2}')

In [ ]:
# Aplica a máscara binária (mantém apenas onde a máscara não é 0 ou null)
irrig_1990_masked = irrig_1990.updateMask(municipalities)
irrig_2020_masked = irrig_2020.updateMask(municipalities)

In [ ]:
# Cria o mapa
Map = geemap.Map(center=[-12.35, -45.53], zoom=9)
Map.addLayer(irrig_1990_masked, {"min": 0, "max": 1, "palette": ['white', 'blue']}, f"Irrigation {year1}")
Map.addLayer(irrig_2020_masked, {"min": 0, "max": 1, "palette": ['white', 'blue']}, f"Irrigation {year2}")
Map

Map(center=[-12.35, -45.53], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [ ]:
# Região de exportação (toda a geometria da máscara)
region = municipalities.geometry()

In [ ]:
# Export the image
geemap.ee_export_image(
    irrig_1990_masked.toByte().selfMask(),  # The image to export
    filename="/content/drive/MyDrive/GEOG347/projeto/irrig_1990_masked.tif",  # The output path on your Google Drive
    scale=30,  # The resolution in meters
    region=municipalities.geometry(),  # The region to export
    file_per_band=False  # Export all bands as a single file
)

Generating URL ...
An error occurred while downloading.
Total request size (367392126 bytes) must be less than or equal to 50331648 bytes.


# **Moran**

In [ ]:
def compute_moran_from_binary_raster(path):
    # Load raster
    with rasterio.open(path) as src:
        raster = src.read(1)

    # Flatten and clean
    valid = ~np.isnan(raster)
    values = raster[valid].flatten()

    # Create spatial weights
    rows, cols = raster.shape
    w = lat2W(rows, cols)
    w_valid = w.subset(valid.flatten())

    # Compute Moran's I
    m = Moran(values, w_valid)
    return m.I, m.p_sim

In [ ]:
m_t0 = compute_moran_from_binary_raster("pivot_t0.tif")
m_t1 = compute_moran_from_binary_raster("pivot_t1.tif")